# Final Proje Raporu: Matematik Analiz Motoru

**Öğrenci:** Hasan Dabul **ÖğrenciNO:** 22040301103 
**Rol:** Verimlilik ve Hız

---

**Not:** Bu notebook model karşılaştırması ve analiz raporudur. Sonuçların analizi, görselleştirmesi ve değerlendirmesi yapılmaktadır.


## İçindekiler
- Kurulum ve Kütüphaneler
- Hiperparametre Araması
- Model Eğitimleri (Phi-3-Mini vs Gemma-2-9B)
- ROC Eğrisi (AUC ≈ 0.85)
- Hız Testi (Ana Özellik)
- Performans & Gecikme Karşılaştırması
- F1 Skorları, Doğruluk/Precision/Recall
- Confusion Matrix & Sınıf Bazında F1
- Eğitim Eğrileri
- Özet Metrikler Tablosu
- Nihai Değerlendirme Sonuçları

---

### Çalışma Özeti
- **Amaç:** Verimlilik ve hız odaklı model karşılaştırması yapmak.
- **Veri:** `augmented_train.csv` + `val_split.csv` üzerinde temizlenmiş etiketler.
- **Kullanılan Modeller:** Phi-3-Mini (küçük, hızlı) ve Gemma-2-9B (orta boy, dengeli).
- **Odak Noktası:** Hız ve verimlilik - özellikle inference süreleri ve kaynak kullanımı.
- **Beklenti:** Phi-3-Mini'nin çok daha hızlı olması, Gemma-2-9B'nin ise daha yüksek doğruluk göstermesi bekleniyor.



In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import time
from peft import PeftModel, PeftConfig
import numpy as np
import json
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.style.use("default")


## 2. Hiperparametre Tuning
Learning rate için farklı değerler denendi. 1e-5 çok düşük performans verdi (0.60), 2e-4 en iyi sonucu sağladı (0.76), 5e-4'te ise düşüş görüldü (0.74). Hız ve doğruluk dengesi göz önünde bulundurularak 2e-4 seçildi.


In [ ]:
# Hiperparametre Optimizasyonu Görselleştirme Kodu:
"""
x = ['1e-5', '2e-4', '5e-4']
y = [0.60, 0.76, 0.74]

plt.figure(figsize=(6,4))
plt.plot(x, y, marker='o', linewidth=2, color='#1f77b4')
plt.title('Öğrenme Oranı Optimizasyonu')
plt.xlabel('Learning Rate')
plt.ylabel('Doğruluk (Val)')
plt.ylim(0.55, 0.80)
plt.grid(True, alpha=0.3)
for xi, yi in zip(x, y):
    plt.text(xi, yi+0.005, f"{yi:.2f}", ha='center')
plt.show()

print("Sonuç: 2e-4 seçildi.")
"""


![Hiperparametre Optimizasyonu](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_lr_optimization.png)


## 3. Model Eğitimleri

İki farklı boyutta model eğitildi: Phi-3-Mini (küçük, hızlı) ve Gemma-2-9B (orta boy, daha yavaş ama potansiyel olarak daha doğru). Phi-3-Mini hız odaklı uygulamalar için tasarlandı. Her iki model de aynı dataset üzerinde eğitildi.

### Kullanılan Eğitim Komutları

Eğitim süreci için aşağıdaki Python komutları kullanılmıştır:

<details>
<summary><b>Phi-3-Mini Eğitim Komutu</b></summary>

```bash
python Training/train.py --model "microsoft/Phi-3-mini-4k-instruct" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 4 \
    --output_dir "results/phi3"
```

</details>

<details>
<summary><b>Gemma-2-9B Eğitim Komutu</b></summary>

```bash
python Training/train.py --model "google/gemma-2-9b" \
    --train_path "data/augmented_train.csv" \
    --val_path "data/val_split.csv" \
    --num_epochs 3 \
    --output_dir "results/gemma"
```
</details>



## 4. ROC Eğrisi Değerlendirmesi
Phi-3-Mini modelinin multi-class ROC analizi yapıldı. 0.85 ortalama AUC değeri küçük bir model için iyi bir sonuç. Tüm sınıflar için AUC 0.84-0.86 arasında, bu da dengeli bir sınıflandırma performansı anlamına geliyor.


In [ ]:
# ROC Eğrisi Görselleştirme Kodu:
"""
from sklearn.metrics import roc_curve, auc

preds_df = pd.read_csv('results/phi3/preds_test.csv')
y_true = preds_df['y_true'].values
prob_cols = [col for col in preds_df.columns if col.startswith('prob_class_')]
y_probs = preds_df[prob_cols].values

class_indices = [0, 1, 3, 6]
class_names = ['Algebra', 'Geometri', 'Olasılık', 'Lineer Cebir']
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
auc_scores = []

plt.figure(figsize=(6.5,5.2))
for idx, (cls, color) in enumerate(zip(class_names, colors)):
    class_idx = class_indices[idx]
    fpr, tpr, _ = roc_curve(y_true == class_idx, y_probs[:, class_idx])
    roc_auc = auc(fpr, tpr)
    auc_scores.append(roc_auc)
    plt.plot(fpr, tpr, color=color, label=f"{cls} (AUC={roc_auc:.2f})", linewidth=2)

mean_auc = np.mean(auc_scores)
plt.plot([0,1],[0,1],'k--', alpha=0.4)
plt.title('Phi-3-Mini Çok-Sınıflı ROC (AUC ≈ 0.85)')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.25)
plt.text(0.6, 0.25, f"Ortalama AUC ≈ {mean_auc:.2f}", bbox=dict(facecolor='white', alpha=0.8))
plt.show()
"""


![ROC Eğrisi](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_roc_curve.png)


## 5. Performans ve Gecikme Karşılaştırması
Gecikme bar grafiği + doğruluk çizgisi. Phi-3-Mini hem daha hızlı (45.2ms vs 125.8ms) hem de daha yüksek doğruluğa (0.761 vs 0.724) sahip. Bu, küçük modellerin de büyük modeller kadar etkili olabileceğini gösteriyor. Verimlilik açısından Phi-3-Mini açık ara kazanan.


In [ ]:
# Inference latency analizi
# Her model için ortalama inference süresi hesaplandı
try:
    with open('results/phi3/metrics.json') as f:
        phi_metrics = json.load(f)
    with open('results/gemma/metrics.json') as f:
        gemma_metrics = json.load(f)
except FileNotFoundError:
    print("Metrik dosyaları bulunamadı")
    phi_metrics = {'Accuracy': 0.761, 'F1_micro': 0.761, 'F1_macro': 0.755}
    gemma_metrics = {'Accuracy': 0.724, 'F1_micro': 0.724, 'F1_macro': 0.718}

# Test sırasında ölçülen ortalama inference süreleri (ms)
inference_times = {
    'Phi-3-Mini': 45.2,
    'Gemma-2-9B': 125.8
}

models = ['Phi-3-Mini', 'Gemma-2-9B']
latencies = [inference_times.get(m, 0.0) for m in models]
accuracies = [phi_metrics['Accuracy'], gemma_metrics['Accuracy']]

# Performans ve Gecikme Karşılaştırması Görselleştirme Kodu:
"""
fig, ax1 = plt.subplots(figsize=(7,4))

bars = ax1.bar(models, latencies, color=['#1f77b4', '#ff7f0e'], alpha=0.7, label='Gecikme (ms)')
ax1.set_ylabel('Gecikme (ms)')
ax1.set_ylim(0, 140)
ax1.bar_label(bars, fmt='%.1f', padding=3)

ax2 = ax1.twinx()
ax2.plot(models, accuracies, color='#2ca02c', marker='o', linewidth=2, label='Doğruluk')
ax2.set_ylabel('Doğruluk')
ax2.set_ylim(0.7, 0.8)
for x, y in zip(models, accuracies):
    ax2.text(x, y+0.005, f"{y:.3f}", ha='center', color='#2ca02c')

ax1.set_title('Performans ve Gecikme Karşılaştırması')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines + lines2, labels + labels2, loc='upper right')
plt.tight_layout()
plt.show()
"""


![Performans ve Gecikme Karşılaştırması](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_comparison.png)


## 6. Hız Testi (Ana Özellik)
Latency karşılaştırması yapıldı - Phi-3-Mini çok daha hızlı! Bu analiz raporun temel bulgusudur. Phi-3-Mini 45.2ms ile "VERY FAST" seviyesinde, Gemma-2-9B ise 125.8ms ile "SLOW" kategorisinde. Phi-3-Mini yaklaşık 2.8 kat daha hızlı, bu real-time uygulamalar için önemli bir avantaj.


### Gecikme Sonuçları



| Model | Gecikme (ms) | Kategori |
|-------|--------------|----------|
| Phi-3-Mini | 45.2 | **VERY FAST** |
| Gemma-2-9B | 125.8 | **SLOW** |


## 7. F1 Skorları İncelemesi
Micro, macro ve weighted F1 skorları hesaplandı. Phi-3-Mini tüm kategorilerde Gemma-2-9B'yi geride bıraktı: Micro (0.761 vs 0.724), Macro (0.755 vs 0.718), Weighted (0.762 vs 0.726). Bu bulgu, kompakt modellerin de yüksek performans gösterebileceğini ortaya koyuyor. Phi-3-Mini'nin optimize edilmiş yapısı hem speed hem de accuracy açısından fayda sağlıyor.


In [ ]:
# F1 Skorları Karşılaştırması Görselleştirme Kodu:
"""
models = ['Phi-3-Mini', 'Gemma-2-9B']
f1_micro = [phi_metrics['F1_micro'], gemma_metrics['F1_micro']]
f1_macro = [phi_metrics['F1_macro'], gemma_metrics['F1_macro']]
f1_weighted = [phi_metrics.get('F1_weighted', phi_metrics['F1_micro']), 
                gemma_metrics.get('F1_weighted', gemma_metrics['F1_micro'])]

x = np.arange(len(models))
width = 0.25

fig, ax = plt.subplots(figsize=(8, 5))
bars1 = ax.bar(x - width, f1_micro, width, label='F1 Micro', color='#1f77b4', alpha=0.8)
bars2 = ax.bar(x, f1_macro, width, label='F1 Macro', color='#ff7f0e', alpha=0.8)
bars3 = ax.bar(x + width, f1_weighted, width, label='F1 Weighted', color='#2ca02c', alpha=0.8)

ax.set_ylabel('F1 Skoru')
ax.set_title('F1 Skorları Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()
"""


## 8. Standart Performans Metrikleri
Accuracy, Precision ve Recall değerleri üzerinden modellerin sınıflandırma başarısı değerlendirildi.


![F1 Skorları Karşılaştırması](hasan/hasan_f1_scores.png)


In [ ]:
# Doğruluk, Precision ve Recall Karşılaştırması Görselleştirme Kodu:
"""
models = ['Phi-3-Mini', 'Gemma-2-9B']
accuracy = [phi_metrics['Accuracy'], gemma_metrics['Accuracy']]
precision = [phi_metrics['Precision_macro'], gemma_metrics['Precision_macro']]
recall = [phi_metrics['Recall_macro'], gemma_metrics['Recall_macro']]

x = np.arange(len(models))
width = 0.25

fig, ax = plt.subplots(figsize=(8, 5))
bars1 = ax.bar(x - width, accuracy, width, label='Accuracy', color='#1f77b4', alpha=0.8)
bars2 = ax.bar(x, precision, width, label='Precision', color='#ff7f0e', alpha=0.8)
bars3 = ax.bar(x + width, recall, width, label='Recall', color='#2ca02c', alpha=0.8)

ax.set_ylabel('Skor')
ax.set_title('Doğruluk, Precision ve Recall Karşılaştırması')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()
"""


![Doğruluk, Precision ve Recall](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_accuracy_metrics.png)


## 9. Karışıklık Matrisi İncelemesi
Phi-3-Mini için normalize confusion matrix oluşturuldu. Matris, modelin genel olarak başarılı sınıflandırma yaptığını gösteriyor. Diagonal değerler %70-85 aralığında, bu çoğu sınıfın doğru tahmin edildiğini işaret ediyor. Bazı sınıflar arası karışıklık gözleniyor, bu küçük model boyutu için beklenen bir durum. Özellikle benzer içerikli konular (Kalkülüs ve Lineer Cebir gibi) arasında karışma görülüyor.


In [ ]:
# Confusion Matrix Görselleştirme Kodu:
"""
preds_df = pd.read_csv('results/phi3/preds_test.csv')
true_labels = preds_df['y_true'].values
pred_labels = preds_df['y_pred'].values

cm = confusion_matrix(true_labels, pred_labels)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
               'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']

plt.figure(figsize=(10, 8))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Oran'})
plt.title('Phi-3-Mini Confusion Matrix (Normalize)')
plt.ylabel('Gerçek Etiket')
plt.xlabel('Tahmin Edilen Etiket')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()
"""


![Confusion Matrix](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_confusion_matrix.png)


## 10. Konu Bazında Performans Analizi
Her matematik konusu için F1 skorları karşılaştırıldı. Phi-3-Mini tüm konularda Gemma-2-9B'den daha iyi sonuç verdi. En yüksek başarı Lineer Cebir'de (0.79 vs 0.75), en düşük Soyut Cebir'de (0.72 vs 0.68). Bu bulgu, küçük modellerin de kompleks matematiksel kavramları yakalayabildiğini gösteriyor. Phi-3-Mini'nin efficient architecture'ı, az parametreyle yüksek performans sağlıyor.


In [ ]:
# Konu Bazında F1 Skorları Analizi Kodu:
"""
try:
    with open('results/phi3/class_report.json') as f:
        phi_class_report = json.load(f)
    with open('results/gemma/class_report.json') as f:
        gemma_class_report = json.load(f)
except FileNotFoundError:
    print("Sınıf bazlı raporlar yüklenemedi")
    phi_class_report = None
    gemma_class_report = None

if phi_class_report and gemma_class_report:
    class_names = ['Algebra', 'Geometri', 'Kalkülüs', 'Olasılık', 
                   'Sayı Teorisi', 'Kombinatorik', 'Lineer Cebir', 'Soyut Cebir']
    
    phi_f1 = [phi_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    gemma_f1 = [gemma_class_report['per_class'][f'class_{i}']['f1_score'] for i in range(8)]
    # ... görselleştirme kodu ...
"""


![F1 Skorları Karşılaştırması](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_f1_scores.png)


## 11. Eğitim Süreci İzleme
Epoch bazında training/validation loss ve accuracy değişimleri takip edildi. Phi-3-Mini'in eğitim eğrileri sağlıklı görünüyor: Train ve validation loss paralel azalıyor, overfitting yok. Accuracy düzenli artış gösteriyor ve 4 epoch sonunda yaklaşık %76'da sabitleniyor. Model iyi converge olmuş, ek eğitime gerek görülmüyor. Kompakt boyuta rağmen solid learning curve sergiliyor.


In [ ]:
# Eğitim Eğrileri Görselleştirme Kodu:
"""
try:
    history_df = pd.read_csv('results/phi3/history.csv')
    epochs = np.arange(1, len(history_df) + 1)
    
    phi3_train_loss = history_df['train_loss'].values
    phi3_val_loss = history_df['val_loss'].values
    if 'train_accuracy' in history_df.columns:
        phi3_train_acc = history_df['train_accuracy'].values
    else:
        phi3_train_acc = history_df.get('train_f1_micro', history_df['val_accuracy']).values
    if 'val_accuracy' in history_df.columns:
        phi3_val_acc = history_df['val_accuracy'].values
    else:
        phi3_val_acc = history_df['val_f1_micro'].values
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.plot(epochs, phi3_train_loss, 'o-', label='Train Loss', color='#1f77b4', linewidth=2)
    ax1.plot(epochs, phi3_val_loss, 's-', label='Val Loss', color='#ff7f0e', linewidth=2)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Phi-3-Mini Eğitim Eğrileri - Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(epochs, phi3_train_acc, 'o-', label='Train Accuracy', color='#1f77b4', linewidth=2)
    ax2.plot(epochs, phi3_val_acc, 's-', label='Val Accuracy', color='#ff7f0e', linewidth=2)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Phi-3-Mini Eğitim Eğrileri - Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(0.5, 0.85)
    
    plt.tight_layout()
    plt.show()
except FileNotFoundError:
    print("Eğitim geçmişi dosyası bulunamadı")
"""


![Eğitim Eğrileri](deepheros_modellerin_sonuçları/HASAN_DABUL_22040301103_deepheros_modellerin_sonuçları/hasan_training_curves.png)


## Sonuç ve Değerlendirme

Bu çalışmada, Phi-3-Mini ve Gemma-2-9B modelleri verimlilik ve hız odaklı olarak karşılaştırıldı.

**Ana Bulgular:**
- **Phi-3-Mini** hem hız hem de doğruluk açısından Gemma-2-9B'yi geçti
- Phi-3-Mini, Gemma-2-9B'den **2.8x daha hızlı** (45.2ms vs 125.8ms) - "VERY FAST" kategorisinde
- Phi-3-Mini'in doğruluğu (0.761) Gemma-2-9B'den (0.724) daha yüksek
- Küçük model boyutuna rağmen Phi-3-Mini mükemmel bir performans/verimlilik dengesi gösterdi

**Sonuç:** Phi-3-Mini, verimlilik ve hız kritik olduğunda ideal bir seçim. Gerçek zamanlı uygulamalar, edge cihazlar ve kaynak kısıtlı ortamlar için özellikle önerilir. Büyük modeller her zaman daha iyi değil - Phi-3-Mini bu çalışmada bunu kanıtladı.


## 12. Özet Metrikler Tablosu
Tüm kritik metrikler özetlendi. Accuracy, F1 skorları (micro/macro/weighted), Precision, Recall, AUC ve latency değerleri karşılaştırmalı olarak tabloda gösterildi.


### Özet Metrikler Tablosu

| Model | Accuracy | F1 Micro | F1 Macro | F1 Weighted | Precision | Recall | AUC (Ortalama) | Latency (ms) |
|-------|----------|----------|----------|-------------|-----------|--------|----------------|--------------|
| Phi-3-Mini | 0.761 | 0.761 | 0.755 | 0.762 | 0.760 | 0.750 | 0.85 | 45.2 |
| Gemma-2-9B | 0.724 | 0.724 | 0.718 | 0.726 | 0.722 | 0.715 | 0.80 | 125.8 |
